In [1]:
%env PYTHONHASHSEED=0

env: PYTHONHASHSEED=0


In [2]:
#Import all the dependencies
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize

import os
import latex
import spacy
import itertools
import scipy
import networkx as nx
import pandas as pd

import financial_data_api as fd
import numpy as np
import matplotlib.pyplot as plt 

from matplotlib import rc
import matplotlib
pgf_with_rc_fonts = {"pgf.texsystem": "pdflatex"}
matplotlib.rcParams.update(pgf_with_rc_fonts)
from gensim.models.doc2vec import Doc2Vec
from sklearn.metrics.pairwise import cosine_similarity


financial_data = fd.FinancialData()

In [3]:
def cos_sim(a, b):
    """Takes 2 vectors a, b and returns the cosine similarity according 
to the definition of the dot product"""
    dot_product = np.dot(a, b)
    norm_a = np.linalg.norm(a)
    norm_b = np.linalg.norm(b)
    return dot_product / (norm_a * norm_b)


In [4]:
def get_outputfile_from_name(model_name, metric, parent_directory):
    fn = model_name + '_' + metric.replace(' ', '_')
    f = os.path.join(parent_directory, fn)
    return f + '.png'

In [5]:
def get_tickers_in_directory():
    return ['AAPL', 'CSCO', 'GOOGL', 'INTC', 'MCD', 'NFLX', 'TRV', 'VZ', 'AXP', 'CVX', 'GS', 'JNJ', 'MMM', 'NKE', 'UNH', 'WBA', 'BA', 'DIS', 'HD', 'JPM', 'MRK', 'PFE', 'UTX', 'WMT', 
           'CAT', 'DOW', 'IBM', 'KO', 'MSFT', 'PG', 'V', 'XOM']

In [6]:
def get_paths_from_tickers(tickers):
    root_data_dir = 'article_data'
    data_directories = [os.path.join(root_data_dir, t) for t in tickers]
    return data_directories

In [7]:
def get_articles_for_tickers(tickers):
    path_names = get_paths_from_tickers(tickers)
    articles = read_articles_from_disk(path_names)
    return set(articles)

In [8]:
def read_articles_from_disk(data_directories):
    counter = 0
    articles = []
    for directory in data_directories: 
        file_list = os.listdir(directory)
        files_to_read = [os.path.join(directory, file) for file in file_list]
        for file in files_to_read: 
            with open(file, 'r') as content_file:
                content = content_file.read()
                articles.append(content)
    return articles

In [9]:
def get_all_sentences(articles):
    nlp = spacy.load("en_core_web_sm")
    all_sentences = []
    counter = 0
    
    for a in articles:
        print('Counter at: ', counter)
        doc = nlp(a)
        for s in doc.sents:
            all_sentences.append(s)
        counter += 1
    return all_sentences


In [10]:
def get_financial_metric_dict(tickers, metric):
    result_dict = {}
    for t in tickers: 
        data = financial_data.get_quarterly_data(t)
        if t not in result_dict:
            if data:
                result_dict[t] = float(data[metric])
    return result_dict

In [11]:
def get_k_closest_neighbors(company_graph, node_source, num_neighbors):
    neighbors = nx.shortest_path_length(company_graph, source=node_source, weight='weight')
    nearest = []
    counter = 0
    print('==============')
    print('Target Node is: ', node_source)

    for n in neighbors:
        print('Neighbor is: ', n)
        nearest.append(n)
        if counter == num_neighbors:
            break
        counter += 1
    print('==============')
    return nearest[1:]

In [12]:
def get_company_set(article, tickers):
    words_found = set([])
    ticker_set = set(tickers)
    article = article.split()
    for word in article:
        if word in ticker_set: 
            words_found.add(word)
    return words_found
 

In [13]:
def get_sp500_list():
    companies = [s for s in pd.read_csv('constituents.csv', header=0)['Symbol']]
    companies.remove('A')
    companies.remove('T')
    companies.remove('GOOG')
    return ['MMM', 'AXP', 'AAPL', 'BA', 'CAT', 'CVX', 'CSCO', 'KO', 'DOW', 'XOM', 'GS', 'HD', 'IBM', 'INTC', 'JNJ', 'JPM',
'MCD', 'MRK', 'MSFT', 'NKE', 'PFE', 'PG', 'TRV', 'UNH', 'UTX', 'VZ', 'V', 'WMT', 'WBA', 'DIS', 'GOOGL', 'NFLX']

In [14]:
def get_occurrence_dict(articles):
    counts_dict = {}
    article_counter = 0
    sp_500_list  = get_sp500_list()
    for a in all_articles:
        company_set = get_company_set(a, sp_500_list)
        pairs = itertools.combinations(company_set, 2)
        pairs = set([tuple(sorted(list(p))) for p in pairs])
        for p in pairs: 
            if p not in counts_dict:
                counts_dict[p] = 0
            counts_dict[p] += 1
    return counts_dict  

In [15]:
def build_embedding_model(sentences, max_epochs, vec_size, alpha, name):
    data = [str(s) for s in sentences]
    tagged_data = [TaggedDocument(words=word_tokenize(_d.lower()), tags=[str(i)]) for i, _d in enumerate(data)]
    
    
    #max_epochs = 100
    #vec_size = 20
    #alpha = 0.025

    model = Doc2Vec(size=vec_size,
                    alpha=alpha, 
                    min_alpha=0.00025,
                    min_count=1,
                    dm =1)

    model.build_vocab(tagged_data)

    for epoch in range(max_epochs):
        print('iteration {0}'.format(epoch))
        model.train(tagged_data,
                    total_examples=model.corpus_count,
                    epochs=model.iter)
        # decrease the learning rate
        model.alpha -= 0.0002
        # fix the learning rate, no decay
        model.min_alpha = model.alpha

    model.save(name)
    print("Model Saved: ", name)
    

In [16]:
def get_average_embeddings(sentences, inspect_set, model):
    vec_size = len(model.docvecs['1'])
    embedding_dict = {t:{'embedding_sum': np.zeros(vec_size), 'count':0} for t in inspect_set}
    for s in sentences: 
        s_str = str(s).split()
        model.random.seed(0)
        s_str_embedding = model.infer_vector(word_tokenize(str(s).lower()))
        for key in embedding_dict:
            if key in s_str:
                embedding_dict[key]['embedding_sum'] += s_str_embedding
                embedding_dict[key]['count'] += 1
    result_dict = {}
    for k, v in embedding_dict.items():
        result_dict[k] = v['embedding_sum']
        if v['count'] > 0: 
            result_dict[k] /= v['count']
    return result_dict



In [17]:
def get_doc2vec_graph(name, f_metric, sentences):
    model= Doc2Vec.load("d2v.model")
    inspect_set = get_sp500_list()
    #embedding_vectors = get_average_embeddings(sentences, inspect_set, model)
    pairs = itertools.combinations(inspect_set, 2)
    ranked_scores = []

    for p in pairs:
        company_A = p[0]
        company_B = p[1]
        
        model.random.seed(0)
        vec_A = model.infer_vector([company_A])
        model.random.seed(0)
        vec_B = model.infer_vector([company_B])
        
        #vec_A = embedding_vectors[company_A].reshape(1, -1)
        #vec_B = embedding_vectors[company_B].reshape(1, -1)
        vec_A = vec_A.reshape(1, -1)
        vec_B = vec_B.reshape(1, -1)

        #cosine_distance = scipy.spatial.distance.cosine(vec_A, vec_B)
        cosine_distance = 1 - cosine_similarity(vec_A, vec_B)[0][0]
        ranked_scores.append((company_A, company_B, cosine_distance))
    embedding_graph = nx.Graph()
    embedding_graph.add_nodes_from(inspect_set)

    embedding_graph.nodes()
    
    for rs in ranked_scores: 
        company_A = rs[0]
        company_B = rs[1]
        distance = rs[2]
        print(company_A, company_B, distance)
        embedding_graph.add_edge(company_A, company_B, weight=distance)
    
    k = 3
    ebit_data = get_financial_metric_dict(inspect_set, f_metric)
    results = []
    n_dict = {}
    for node in embedding_graph.nodes():
        neighbors = get_k_closest_neighbors(embedding_graph, node, k)
        neighbor_stats = []
        n_dict[node] = neighbors

        for n in neighbors: 
            if n in ebit_data:
                neighbor_stats.append(ebit_data[n])
        deviation = np.std(neighbor_stats)
        results.append((node, deviation))
    save_plots_for_results(results, name, f_metric)
    return results
    

In [18]:
def get_cooccurrence_graph(all_articles, name, f_metric):
    sp_500_list = get_sp500_list()
    co_occurrence_dict = get_occurrence_dict(all_articles)
    node_set = set([])
    for pair in co_occurrence_dict:
        node_set.add(pair[0])
        node_set.add(pair[1])

    co_occurrence_graph = nx.Graph()
    co_occurrence_graph.add_nodes_from(list(node_set))

    max_score = max([v for v in co_occurrence_dict.values()])

    for pair, score in co_occurrence_dict.items():
        company_A = pair[0]
        company_B = pair[1]
        co_occurrence_graph.add_edge(company_A, company_B, weight=(1 - (score/max_score)))
    k = 3  
    ebit_data = get_financial_metric_dict(node_set, f_metric)
    results_baseline = []
    n_dict = {}
    for node in co_occurrence_graph.nodes():
        neighbors = get_k_closest_neighbors(co_occurrence_graph, node, k)
        neighbor_stats = []
        n_dict[node] = neighbors
        for n in neighbors: 
            if n in ebit_data:
                neighbor_stats.append(ebit_data[n])
        deviation = np.std(neighbor_stats)
        results_baseline.append((node, deviation))
    save_plots_for_results(results_baseline, name, f_metric)
    return results_baseline

In [19]:
def save_plots_for_results(results, output_file, metric):
    top = sorted(results, key=lambda x:x[1])

    labels, ys = zip(*top)
    xs = np.arange(len(labels)) 
    width = 1
    #plt.rc('text', usetex=True)

    plt.bar(xs, ys, width, align='center')

    plt.xticks(xs, labels, rotation='vertical') #Replace default x-ticks with xs, then replace xs with labels
    #plt.yticks(ys)
    plt.xlabel('Company')
    plt.ylabel('Standard Deviation')
    plt.title('Neighborhood STD for ' + metric)

    plt.savefig(output_file, dpi=300)
    plt.clf()

In [20]:
tickers = get_tickers_in_directory()
all_articles = get_articles_for_tickers(tickers)
#all_sentences = get_all_sentences(all_articles)
all_sentences = None

In [21]:
metrics_list = [
            'Gross Profit Growth', 
            'EBIT Growth', 
            'Operating Income Growth', 
            'Net Income Growth', 
            'EPS Growth', 
            'EPS Diluted Growth',
            'Weighted Average Shares Growth',
            'Dividends per Share Growth',
            'Operating Cash Flow growth',
            'Free Cash Flow growth',
            'Receivables growth',
            'Inventory Growth',
            'Asset Growth',
            'Book Value per Share Growth',
            'Debt Growth',
            'R&D Expense Growth',
            'SG&A Expenses Growth'
]
for m in metrics_list:
    results = get_doc2vec_graph(get_outputfile_from_name('doc2vec', m, 'output_naive'), m, all_sentences)
    results_baseline = get_cooccurrence_graph(all_articles, get_outputfile_from_name('baseline', m, 'output_naive'), m)

    improved_keys = {r[0]:r[1] for r in results}
    baseline_keys = {r[0]:r[1] for r in results_baseline}
    
    difference_dict = {}
    for key in improved_keys: 
        if key in baseline_keys: 
            difference_dict[key] = improved_keys[key] - baseline_keys[key]
    counter = 0
    for k, v in difference_dict.items():
        if v <= 0: 
            counter += 1
    print(m, counter)
    
    difference_results = [(k, v) for k, v in difference_dict.items()]
    save_plots_for_results(difference_results, get_outputfile_from_name('difference', m, 'output_naive'), m)
    

/anaconda3/lib/python3.7/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


MMM AXP 0.9753306582570076
MMM AAPL 1.0420166999101639
MMM BA 0.6416184604167938
MMM CAT 1.3015573024749756
MMM CVX 0.7301180064678192
MMM CSCO 1.3092803657054901
MMM KO 0.8745367377996445
MMM DOW 0.8042533248662949
MMM XOM 1.0370443426072598
MMM GS 1.2463473081588745
MMM HD 1.1238917335867882
MMM IBM 0.972420196980238
MMM INTC 0.934938095510006
MMM JNJ 1.1915784627199173
MMM JPM 0.7330257892608643
MMM MCD 0.8260805904865265
MMM MRK 0.7860342860221863
MMM MSFT 0.9287530928850174
MMM NKE 0.6999785900115967
MMM PFE 0.6571518778800964
MMM PG 1.065467394888401
MMM TRV 1.1048555672168732
MMM UNH 0.7835893779993057
MMM UTX 1.1741718649864197
MMM VZ 0.9982624299591407
MMM V 1.2500017285346985
MMM WMT 1.0677255615592003
MMM WBA 1.1635762453079224
MMM DIS 1.1384268403053284
MMM GOOGL 0.7784657031297684
MMM NFLX 0.7039743065834045
AXP AAPL 1.0003771665506065
AXP BA 1.0649667233228683
AXP CAT 1.1358548551797867
AXP CVX 1.3559415340423584
AXP CSCO 1.1324719041585922
AXP KO 1.3483005166053772
AXP D

Target Node is:  MMM
Neighbor is:  MMM
Neighbor is:  BA
Neighbor is:  PFE
Neighbor is:  NKE
Target Node is:  AXP
Neighbor is:  AXP
Neighbor is:  NFLX
Neighbor is:  PG
Neighbor is:  HD
Target Node is:  AAPL
Neighbor is:  AAPL
Neighbor is:  GS
Neighbor is:  NKE
Neighbor is:  CSCO
Target Node is:  BA
Neighbor is:  BA
Neighbor is:  MCD
Neighbor is:  MMM
Neighbor is:  INTC
Target Node is:  CAT
Neighbor is:  CAT
Neighbor is:  CSCO
Neighbor is:  XOM
Neighbor is:  INTC
Target Node is:  CVX
Neighbor is:  CVX
Neighbor is:  VZ
Neighbor is:  MMM
Neighbor is:  NKE
Target Node is:  CSCO
Neighbor is:  CSCO
Neighbor is:  GS
Neighbor is:  CAT
Neighbor is:  AAPL
Target Node is:  KO
Neighbor is:  KO
Neighbor is:  MSFT
Neighbor is:  UTX
Neighbor is:  DOW
Target Node is:  DOW
Neighbor is:  DOW
Neighbor is:  GOOGL
Neighbor is:  IBM
Neighbor is:  DIS
Target Node is:  XOM
Neighbor is:  XOM
Neighbor is:  UNH
Neighbor is:  TRV
Neighbor is:  NKE
Target Node is:  GS
Neighbor is:  GS
Neighbor is:  CSCO
Neighbor is

/anaconda3/lib/python3.7/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


MMM AXP 0.9753306582570076
MMM AAPL 1.0420166999101639
MMM BA 0.6416184604167938
MMM CAT 1.3015573024749756
MMM CVX 0.7301180064678192
MMM CSCO 1.3092803657054901
MMM KO 0.8745367377996445
MMM DOW 0.8042533248662949
MMM XOM 1.0370443426072598
MMM GS 1.2463473081588745
MMM HD 1.1238917335867882
MMM IBM 0.972420196980238
MMM INTC 0.934938095510006
MMM JNJ 1.1915784627199173
MMM JPM 0.7330257892608643
MMM MCD 0.8260805904865265
MMM MRK 0.7860342860221863
MMM MSFT 0.9287530928850174
MMM NKE 0.6999785900115967
MMM PFE 0.6571518778800964
MMM PG 1.065467394888401
MMM TRV 1.1048555672168732
MMM UNH 0.7835893779993057
MMM UTX 1.1741718649864197
MMM VZ 0.9982624299591407
MMM V 1.2500017285346985
MMM WMT 1.0677255615592003
MMM WBA 1.1635762453079224
MMM DIS 1.1384268403053284
MMM GOOGL 0.7784657031297684
MMM NFLX 0.7039743065834045
AXP AAPL 1.0003771665506065
AXP BA 1.0649667233228683
AXP CAT 1.1358548551797867
AXP CVX 1.3559415340423584
AXP CSCO 1.1324719041585922
AXP KO 1.3483005166053772
AXP D

...

Target Node is:  MMM
Neighbor is:  MMM
Neighbor is:  BA
Neighbor is:  PFE
Neighbor is:  NKE
Target Node is:  AXP
Neighbor is:  AXP
Neighbor is:  NFLX
Neighbor is:  PG
Neighbor is:  HD
Target Node is:  AAPL
Neighbor is:  AAPL
Neighbor is:  GS
Neighbor is:  NKE
Neighbor is:  CSCO
Target Node is:  BA
Neighbor is:  BA
Neighbor is:  MCD
Neighbor is:  MMM
Neighbor is:  INTC
Target Node is:  CAT
Neighbor is:  CAT
Neighbor is:  CSCO
Neighbor is:  XOM
Neighbor is:  INTC
Target Node is:  CVX
Neighbor is:  CVX
Neighbor is:  VZ
Neighbor is:  MMM
Neighbor is:  NKE
Target Node is:  CSCO
Neighbor is:  CSCO
Neighbor is:  GS
Neighbor is:  CAT
Neighbor is:  AAPL
Target Node is:  KO
Neighbor is:  KO
Neighbor is:  MSFT
Neighbor is:  UTX
Neighbor is:  DOW
Target Node is:  DOW
Neighbor is:  DOW
Neighbor is:  GOOGL
Neighbor is:  IBM
Neighbor is:  DIS
Target Node is:  XOM
Neighbor is:  XOM
Neighbor is:  UNH
Neighbor is:  TRV
Neighbor is:  NKE
Target Node is:  GS
Neighbor is:  GS
Neighbor is:  CSCO
Neighbor is

/anaconda3/lib/python3.7/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


MMM AXP 0.9753306582570076
MMM AAPL 1.0420166999101639
MMM BA 0.6416184604167938
MMM CAT 1.3015573024749756
MMM CVX 0.7301180064678192
MMM CSCO 1.3092803657054901
MMM KO 0.8745367377996445
MMM DOW 0.8042533248662949
MMM XOM 1.0370443426072598
MMM GS 1.2463473081588745
MMM HD 1.1238917335867882
MMM IBM 0.972420196980238
MMM INTC 0.934938095510006
MMM JNJ 1.1915784627199173
MMM JPM 0.7330257892608643
MMM MCD 0.8260805904865265
MMM MRK 0.7860342860221863
MMM MSFT 0.9287530928850174
MMM NKE 0.6999785900115967
MMM PFE 0.6571518778800964
MMM PG 1.065467394888401
MMM TRV 1.1048555672168732
MMM UNH 0.7835893779993057
MMM UTX 1.1741718649864197
MMM VZ 0.9982624299591407
MMM V 1.2500017285346985
MMM WMT 1.0677255615592003
MMM WBA 1.1635762453079224
MMM DIS 1.1384268403053284
MMM GOOGL 0.7784657031297684
MMM NFLX 0.7039743065834045
AXP AAPL 1.0003771665506065
AXP BA 1.0649667233228683
AXP CAT 1.1358548551797867
AXP CVX 1.3559415340423584
AXP CSCO 1.1324719041585922
AXP KO 1.3483005166053772
AXP D

Target Node is:  MMM
Neighbor is:  MMM
Neighbor is:  BA
Neighbor is:  PFE
Neighbor is:  NKE
Target Node is:  AXP
Neighbor is:  AXP
Neighbor is:  NFLX
Neighbor is:  PG
Neighbor is:  HD
Target Node is:  AAPL
Neighbor is:  AAPL
Neighbor is:  GS
Neighbor is:  NKE
Neighbor is:  CSCO
Target Node is:  BA
Neighbor is:  BA
Neighbor is:  MCD
Neighbor is:  MMM
Neighbor is:  INTC
Target Node is:  CAT
Neighbor is:  CAT
Neighbor is:  CSCO
Neighbor is:  XOM
Neighbor is:  INTC
Target Node is:  CVX
Neighbor is:  CVX
Neighbor is:  VZ
Neighbor is:  MMM
Neighbor is:  NKE
Target Node is:  CSCO
Neighbor is:  CSCO
Neighbor is:  GS
Neighbor is:  CAT
Neighbor is:  AAPL
Target Node is:  KO
Neighbor is:  KO
Neighbor is:  MSFT
Neighbor is:  UTX
Neighbor is:  DOW
Target Node is:  DOW
Neighbor is:  DOW
Neighbor is:  GOOGL
Neighbor is:  IBM
Neighbor is:  DIS
Target Node is:  XOM
Neighbor is:  XOM
Neighbor is:  UNH
Neighbor is:  TRV
Neighbor is:  NKE
Target Node is:  GS
Neighbor is:  GS
Neighbor is:  CSCO
Neighbor is

<Figure size 432x288 with 0 Axes>